# How to track token usage for LLMs

Tracking token usage to calculate cost is an important part of putting your app in production. This guide goes over how to obtain this information from your LangChain model calls.

```{=mdx}
import PrerequisiteLinks from "@theme/PrerequisiteLinks";

<PrerequisiteLinks content={`
- [LLMs](/docs/concepts/#llms)
`} />
```

## Using LangSmith

You can use [LangSmith](https://www.langchain.com/langsmith) to help track token usage in your LLM application. See the [LangSmith quick start guide](https://docs.smith.langchain.com/).

## Using callbacks

There are some API-specific callback context managers that allow you to track token usage across multiple calls. You'll need to check whether such an integration is available for your particular model.

If such an integration is not available for your model, you can create a custom callback manager by adapting the implementation of the [OpenAI callback manager](https://api.python.langchain.com/en/latest/_modules/langchain_community/callbacks/openai_info.html#OpenAICallbackHandler).

### OpenAI

Let's first look at an extremely simple example of tracking token usage for a single Chat model call.

:::{.callout-danger}

The callback handler does **NOT** work when **streaming** becauyse OpenAI does not return token information in its API response.

::

### Single call

In [1]:
from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

with get_openai_callback() as cb:
    result = llm.invoke("Tell me a joke")
    print(result)
    print("---")
print()

print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")


Why don't scientists trust atoms?

Because they make up everything!
---

Total Tokens: 18
Prompt Tokens: 4
Completion Tokens: 14
Total Cost (USD): $3.4e-05


### Multiple calls

Anything inside the context manager will get tracked. Here's an example of using it to track multiple calls in sequence to a chain. This will also work for an agent which may use multiple steps.

In [50]:
from langchain_community.callbacks import get_openai_callback
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

template = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = template | llm

with get_openai_callback() as cb:
    response = chain.invoke({"topic": "birds"})
    print(response)
    response = chain.invoke({"topic": "fish"})
    print("--")
    print(response)


print()
print("---")
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")



Why did the chicken go to the seance?

To talk to the other side of the road!
--


Why did the fish blush?

Because it saw the ocean's bottom!

---
Total Tokens: 49
Prompt Tokens: 12
Completion Tokens: 37
Total Cost (USD): $9.200000000000001e-05


## Streaming

:::{.callout-danger}

`get_openai_callback` relies on metadata information in the API response to get
information about the token counts; however, the `OpenAI` API does **NOT** return
token counts when streaming.

If you want to count token counts correctly, for streaming, either
implement a custom callback handler that uses an appropriate tokenizers to
count the tokens or use a monitoring platform like [LangSmith](https://www.langchain.com/langsmith).
:::

In [7]:
from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

with get_openai_callback() as cb:
    for chunk in llm.stream("Tell me a joke"):
        print(chunk, end="", flush=True)
    print(result)
    print("---")
print()

print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")



Why couldn't the bicycle stand up by itself?

Because it was two-tired!

Why don't scientists trust atoms?

Because they make up everything.
---

Total Tokens: 0
Prompt Tokens: 0
Completion Tokens: 0
Total Cost (USD): $0.0
